In [35]:
import numpy as np
import pandas as pd
import cv2
import matplotlib as plt
from utils.quaternion import *

In [7]:
xsense = pd.read_csv("input/Participant_541_Setup_A_Seq_5_Trial_3.xsens.csv").iloc[1:]

In [8]:
xsense_orig = xsense.copy()
xsense = xsense[::10]

In [9]:
xsense.shape

(2338, 743)

In [14]:
xsense.shape

(2338, 743)

In [29]:
xsense_filtered = pd.concat([xsense.filter(regex='orientation', axis=1).copy(), 
                             xsense.filter(regex='position', axis=1).copy()], axis=1)

In [30]:
xsense_filtered.shape

(2338, 161)

In [40]:
float(xsense_filtered.iloc[0]["orientation_Pelvis_q0"])

0.99756

In [48]:
q = np.array([float(xsense_filtered.iloc[0]["orientation_Pelvis_q0"]), float(xsense_filtered.iloc[0]["orientation_Pelvis_q1"]),
             float(xsense_filtered.iloc[0]["orientation_Pelvis_q2"]), 
              float(xsense_filtered.iloc[0]["orientation_Pelvis_q2"])]).reshape(-1, 4)

In [51]:
v = np.array([float(xsense_filtered.iloc[0]["position_Pelvis_x"]), float(xsense_filtered.iloc[0]["position_Pelvis_y"]),
             float(xsense_filtered.iloc[0]["position_Pelvis_z"])]).reshape(-1, 3)

tensor([[ 0.9976,  0.0023, -0.0408, -0.0408]], dtype=torch.float64)

In [54]:
qrot(torch.from_numpy(q), torch.from_numpy(v))

tensor([[0.0203, 0.0791, 1.0872]], dtype=torch.float64)

In [ ]:
 # syncing video according to videos as video is 25 fps

In [ ]:
xsense.shape

In [ ]:
v_cap = cv2.VideoCapture("input/Participant_541_Setup_A_Seq_5_Trial_3.camera1.mp4")
fps = v_cap.get(cv2.CAP_PROP_FPS)
frames = v_cap.get(cv2.CAP_PROP_FRAME_COUNT)
print("frame rate {} and video lenght secs {} ".format(fps, frames // fps))

In [ ]:
# Read until video is completed 
ind = 300
while(v_cap.isOpened()): 
      
    # Capture frame-by-frame 
    ret, frame = v_cap.read() 
    if ret == True: 
        
        frame = cv2.circle(frame, (1000, 700), 10, (255, 0, 0), 3)
        ind += 1
        # Display the resulting frame
        cv2.imshow('Frame', frame) 

        # Press Q on keyboard to  exit 
        if cv2.waitKey(int(fps)) & 0xFF == ord('q'): 
            break

    # Break the loop 
    else:  
        break
        
# When everything done, release  
# the video capture object 
v_cap.release() 
   
# Closes all the frames 
cv2.destroyAllWindows() 

In [ ]:
frame.shape

In [34]:
%%writefile ./utils/__init__.py




Writing ./utils/__init__.py


In [ ]:
%%writefile ./utils/quaternion.py

# Copyright (c) 2018-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import torch

def qrot(q, v):
    """
    Rotate vector(s) v about the rotation described by quaternion(s) q.
    Expects a tensor of shape (*, 4) for q and a tensor of shape (*, 3) for v,
    where * denotes any number of dimensions.
    Returns a tensor of shape (*, 3).
    """
    assert q.shape[-1] == 4
    assert v.shape[-1] == 3
    assert q.shape[:-1] == v.shape[:-1]

    qvec = q[..., 1:]
    uv = torch.cross(qvec, v, dim=len(q.shape)-1)
    uuv = torch.cross(qvec, uv, dim=len(q.shape)-1)
    return (v + 2 * (q[..., :1] * uv + uuv))
    
    
def qinverse(q, inplace=False):
    # We assume the quaternion to be normalized
    if inplace:
        q[..., 1:] *= -1
        return q
    else:
        w = q[..., :1]
        xyz = q[..., 1:]
        return torch.cat((w, -xyz), dim=len(q.shape)-1)